In [41]:
## Necessary installs from anaconda prompt
## pip install imutils
## pip install opencv-python 
## pip3 install tensorflow --user

## you can see description of function using shift+tab
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths ## from anaconda promt "pip install imutils" for installing

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [42]:
dataset = r"C:\Users\hilmi\Desktop\7 semester\bitirme\dataset"
imagePaths = list(paths.list_images(dataset))

In [59]:
data = []
labels =[]
for i in imagePaths:
    label = i.split(os.path.sep)[-2]
    labels.append(label)
    image = load_img(i,target_size=(224,224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

C:\Users\hilmi\anaconda3\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [60]:
data = np.array(data,dtype = "float32")
labels = np.array(labels)

In [61]:
labels

array(['without_mask', 'without_mask', 'without_mask', ..., 'with_mask',
       'with_mask', 'with_mask'], dtype='<U12')

In [62]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [58]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [63]:
train_X,test_X,train_Y,test_Y=train_test_split(data,labels,test_size=0.20,stratify=labels,random_state=10)

In [71]:
train_X.shape, test_X.shape, train_Y.shape, test_Y.shape


((3066, 224, 224, 3), (767, 224, 224, 3), (3066, 2), (767, 2))

In [72]:
aug=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,vertical_flip=True,fill_mode='nearest')

In [75]:
baseModel=MobileNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

In [77]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [78]:
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='Flatten')(headModel)
headModel=Dense(128,activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)

model=Model(inputs=baseModel.input,outputs=headModel)

In [79]:
for layer in baseModel.layers:
    layer.trainable=False

In [80]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [81]:
learning_rate=0.001
Epochs=20
BS=12

opt=Adam(lr=learning_rate,decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

H=model.fit(
    aug.flow(train_X,train_Y,batch_size=BS),
    steps_per_epoch=len(train_X)//BS,
    validation_data=(test_X,test_Y),
    validation_steps=len(test_X)//BS,
    epochs=Epochs
)


model.save(r'C:\Users\hilmi\Desktop\7 semester\bitirme\mobilenet_v2_mask.model')

Epoch 1/20
255/255 [==============================] - 48s 187ms/step - loss: 0.1861 - accuracy: 0.9286 - val_loss: 0.0740 - val_accuracy: 0.9726
Epoch 2/20
255/255 [==============================] - 38s 148ms/step - loss: 0.0985 - accuracy: 0.9623
Epoch 3/20
255/255 [==============================] - 38s 147ms/step - loss: 0.0808 - accuracy: 0.9722
Epoch 4/20
255/255 [==============================] - 38s 148ms/step - loss: 0.0723 - accuracy: 0.9735
Epoch 5/20
255/255 [==============================] - 38s 148ms/step - loss: 0.0612 - accuracy: 0.9820
Epoch 6/20
255/255 [==============================] - 38s 148ms/step - loss: 0.0502 - accuracy: 0.9830
Epoch 7/20
255/255 [==============================] - 38s 148ms/step - loss: 0.0679 - accuracy: 0.9758
Epoch 8/20
255/255 [==============================] - 38s 148ms/step - loss: 0.0583 - accuracy: 0.9797
Epoch 9/20
255/255 [==============================] - 38s 149ms/step - loss: 0.0555 - accuracy: 0.9820
Epoch 10/20
255/255 [==========

In [82]:
predict=model.predict(test_X,batch_size=BS)
predict=np.argmax(predict,axis=1)
print(classification_report(test_Y.argmax(axis=1),predict,target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767

